In [2]:
pip install altair

  Obtaining dependency information for altair from https://files.pythonhosted.org/packages/17/16/b12fca347ff9d062e3c44ad9641d2ec50364570a059f3078ada3a5119d7a/altair-5.1.2-py3-none-any.whl.metadata
   ---------------------------------------- 516.2/516.2 kB 8.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from gurobipy import *
import altair as alt
import math

Input Parameters

In [4]:
#Cost parameters
Cf=1
# cost of mining
C1=2
#C1 - hydrometallurgy production cost ($/t)
C2=3
#C2 - pyrometallurgy production cost ($/t)
Cm=6
#Cm-cost of manufacuring ($/t)
Ct1=4
#Ct1 - copper transportation by EV truck cost ($/t)
Ct2=5
#Ct2 - copper transportation by diesel truck cost ($/t)
CR1=6
#CR1- recycling of end-use to production cost ($/t)
CR2=7
#CR2- recycling of end-use to manufacturing cost ($/t)

In [5]:
# Emission factors 
EMf=4
#EMf - emission factor of mining (t CO2-eq/t)
EM1=2
#EM1 - emission factor of hydrometallurgy production (t CO2-eq/t)
EM2=3
#EM2 - emission factor of pyrometallurgy production (t CO2-eq/t)

EMf= 8
# EMf - emission factor of manufacturing (t CO2-eq/t)
EMt1=4
#EMt1 - emission factor of copper transportation by EV truck (t CO2-eq/t)
EMt2=5
#EMt2 - emission factor of copper transportation by diesel truck (t CO2-eq/t)
EMR1=4
#EMR1- emission factor of recycling of end-use to production (t CO2-eq/t)
EMR2=3
#EMR2- emission factor of recycling of end-use to manufacturing (t CO2-eq/t)


In [6]:
# Energy consumption factor
Ef=8
#Ef - energy consumption of mining (kwh/t)
E1=4
#E1 - energy consumption of hydrometallurgy production cost (kwh/t)
E2=4
#E2 - energy consumption of pyrometallurgy production cost (kwh/t)
Em=9
#Em - energy consumption of manufacturing (kwh/t)
Et1=5
#Et1 - energy consumption of copper transportation by EV truck cost (kwh/t)
Et2 =7
#Et2 - energy consumption of copper transportation by diesel truck cost (kwh/t)
ER1=8
#ER1- energy consumption of recycling of end-use to production cost (kwh/t)
ER2=9
#ER2- energy consumption of recycling of end-use to manufacturing cost (kwh/t)

In [7]:
# Demand and production parameters
D=400
#D is the minimum demand (t)
P=350

#P is the minimum production (t)


In [8]:
m = gp.Model()

Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-07


In [9]:
# Add decision varibles
# f - production volume from mining (t); X1 - hydrometallurgy production (t); X2 - pyrometallurgy
#production (t);
#Y1 - copper transportation by EV truck (t); Y2 - copper transportation by diesel truck (t); R1-
#recycling rate of end-use to production (%)
#R2- recycling rate of end-use to manufacturing (%)

f = m.addVar(name="f")
X1 = m.addVar(name="X1")
X2 = m.addVar(name="X2")
Y1 = m.addVar(name="Y1")
Y2 = m.addVar(name="Y2")
R1 = m.addVar( name="R1")
R2 = m.addVar( name="R2")


In [10]:
# Add constraints
m.addConstr (f == X1+X2)
m.addConstr (Y1+Y2 == X1+X2+(Y1+Y2)*(R1+R2))
m.addConstr ((Y1+Y2)*(R1+R2) >=f )
m.addConstr (f>=P)
m.addConstr (Y1+Y2>=D)
m.addConstr (R1<=0.5)
m.addConstr (R2<=0.5)

<gurobi.Constr *Awaiting Model Update*>

In [11]:
#Objective function
m.setObjective(f*Cf + X1*C1+ X2*C2+ (X1+X2)*Cm + Y1*Ct1+ Y2*Ct2 +(Y1+Y2)*R1*CR1+(Y1+Y2)*R2*CR2, gp.GRB.MINIMIZE)
# minimze cost 

m.setObjective(f*EMf + X1*EM1+ X2*EM2+ (X1+X2)*EMf + Y1*EMt1 + Y2*EMt2 + (Y1+Y2)*
R1*EMR1 + (Y1+Y2) *R2*EMR2, gp.GRB.MINIMIZE)

# minimze emissions

m.setObjective(f*Ef + X1*E1+ X2*E2+ (X1+X2)*Em + Y1*Et1 +Y2*Et2+ (Y1+Y2)*R1*ER1 + (Y1+Y2)*R2*ER2
, gp.GRB.MINIMIZE)

# minimize energy 

In [12]:
m.setParam('NonConvex', 2)
m.optimize()

Set parameter NonConvex to value 2
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Core(TM) i5-10310U CPU @ 1.70GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 5 rows, 7 columns and 8 nonzeros
Model fingerprint: 0xa8a45e5c
Model has 4 quadratic objective terms
Model has 2 quadratic constraints
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [5e+00, 1e+01]
  QObjective range [2e+01, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-01, 4e+02]
Presolve removed 3 rows and 0 columns

Continuous model is non-convex -- solving as a MIP

Presolve removed 3 rows and 0 columns
Presolve time: 0.02s
Presolved: 20 rows, 13 columns, 45 nonzeros
Presolved model has 4 bilinear constraint(s)
Variable types: 13 continuous, 0 integer (0 binary)

Root relaxation: objective 1.3650